# OpenStreetMap Data Case Study
## Map Area
I have chosen the city of Edmonton, Alberta, Canada. I have recently moved to Edmonton and believe this project will help me to know the city better.

The city area in openstreetmap.org: https://www.openstreetmap.org/relation/2564500  
Downloaded the OSM XML file from: https://mapzen.com/data/metro-extracts/metro/edmonton_canada/

## Problems encountered in the map
### Street names
After initial audit of the street names two main inconsistencies were encountered:
- Abbreviated street types: `St` versus `Street`
- Multiple spaces between words: `207 Street   North-west` having 3 spaces between `Street` and `North-west`

### House number
The audit on housenumber revealed that:
- The house number is often part of the street name: `26323 Township Road 532A` where `26323` is the house number. This mostly happens for rural areas

### Postal code
The postal code in Canada is in the form of `A1A 1A1` and usally written in capitals letters.  
Often in the data base the postal codes are written in lower letters or the two parts are attached. 

### City, Province, Country
For `way` tags often city, province, and country infromation was combined in `is_in` tag.

## Fixing the issues
When auditing and fixing the street names the following rules where followed. 
-	If the last word is a direction word (east, north, etc.) the second last word will also be audited 
-	If the last word is a number (e.g. `Township Road 532A`) the second last word will be audited
-	If the last word is a number, and the first word is also a number, then the first word would be considered as the house number
-	The spacing between words will be reduced to 1 space

The postal code was updated to match the standard. i.e. used capital letters and one space between the first three and second three characters. 

The province and country names, when present, were replaced with Alberta and Canada, respectively. If the information was in `is_in` tag, it was changed to appropriate city, province and country tags. 

I used `.split()` to find individual words and check them one by one. I also used regular expressions to check the road number such as `532A`

I updated the `update_name` function in audit.py to the follwoing code:



In [ ]:
expected_position = ["North-west", "South-west", "North-east", "South-east", "North", "South", "East", "West"]
road_num_re = re.compile(r'[0-9]+[a-zA-Z]?\.?$') # combination of numbers with zero or one letter at the end
house_num_re = re.compile(r'[0-9]+[a-zA-Z]?$') # combination of numbers with zero or one letter at the end 

def update_name(name):

    # YOUR CODE HERE
    house_num = None
    street_type = None
    m = name.split()
    if road_num_re.search(m[-1]) and len(m) >= 2: # if last word is road number and more than two words
        street_type = m[-2] #check the second last word 
        if street_type in expected_type:
            if house_num_re.search(m[0]): # if the first word is house number 
                house_num = m[0]
                m = m[1:] # remove the house number from street name
        else:
            if street_type in mapping_type: # if street type can be improved 
                m[-2] = mapping_type[street_type] #update the second last word
    elif m[-1] in expected_position or m[-1] in mapping_position: #if last word is position or from postion mapping dictionary
        if m[-1] in mapping_position: 
            m[-1] = mapping_position[m[-1]]
        street_type = m[-2] #check the second last word
        if street_type in mapping_type:
            m[-2] = mapping_type[street_type] #update the second last word
    else: # otherwise last word is street type
        street_type = m[-1]
        if street_type in mapping_type:
            m[-1] = mapping_type[street_type]
    
    return ' '.join(m), street_type, house_num

the postal code update code is shown below. the code first checks the validity of postal code, and if necessary adds a space between the two parts.

In [ ]:
postcode_re = re.compile(r'[A-Z][0-9][A-Z]\s?[0-9][A-Z][0-9]', re.IGNORECASE)
def update_postcode(code):
    if postcode_re.match(code):
        if code[3] == ' ':
            return code.upper()
        else:
            return code[:3].upper() + ' ' + code[3:6].upper()
    else:
        return code

The code below shows the process to parse the `is_in` tag

In [ ]:
def parse_isin(name):
    address = name.split(',') # the names are separated by ','
    if len(address) == 3 and address[2] == 'Canada': # if three parts and last part 'Canada'
        return address[0], address[1], address[2]
    elif len(address) == 2 and address[1] == 'Alberta': # if two parts and last part 'Alberta'
        return address[0], address[1], None
    elif len(address) >= 3 and address[1] == ' County of': # if the second part is ' County of',
                                                            # this was the case for rural cities 
        city = address[0].split() # only take the first word of the city. this approach is not very robust, 
                                # as two word cities will be missed. But worked fine for current dataset.
        if len(address) == 3:
            return city[0], address[2], None       
        elif len(address) == 4:
            return city[0], address[2], address[3]
    elif len(address) == 1 and address != 'Alberta': # if only one part and not 'Alberta', then it is the city
        return address, None, None      
    return None, None, None


### Outstanding issues
Despite making the fixes mentioned above, numerous minor issues remain that are not easy to programatically  fix.
The instanses that have this type of issues were usually limited and would be easier to fix manually rather than writing a program for it. In this project I chose to skip fixing these. 
- `137 Avenue North West`, here the `North` and `West` are separated with a space and cannot be easily fixed using the approach for other issues
- `87th Avenue`, where the street number is written as an ordinal number
- `87 Avenue and 114 Street`, street address is given as intersection of two streets
- `Rue Monette`, which is the French for `Monette Street` 


### Adding data to the CSV files
In the function `shape_element` in `data.py` file, when the tag key is `street` the function for updating street name will be called. additinally if the house number if not `None` the house number will be added to the database.  
Similarly, the postal code will be updated to the standard form. The province and country will also be set by default to Alberta and Canada. 

In [ ]:
            if dic['key'] == 'street': #update the name if street name
                value, _, house_num = audit.update_name(value)
                if house_num != None:
                    tags.append({'id': node_attribs['id'], 
                                 'key': 'housenumber',
                                 'type': 'addr',
                                 'value': house_num})
            elif dic['key'] == 'postcode':
                value = audit.update_postcode(value)
            elif dic['key'] == 'country':
                value = 'Canada'
            elif dic['key'] == 'province':
                value = 'Alberta'

In the `way` tags, when `is_in` tag is encountered, its information will be parsed and put into appropriate separate tags. This would significantly help with making queries later. 

In [ ]:
            elif key == 'is_in':
                city, province, country = audit.parse_isin(value)
                if city != None:
                    tags.append({'id': way_attribs['id'], 
                                 'key': 'city',
                                 'type': 'is_in',
                                 'value': city})
                if province != None:
                    tags.append({'id': way_attribs['id'], 
                                 'key': 'province',
                                 'type': 'is_in',
                                 'value': province})
                if country != None:
                    tags.append({'id': way_attribs['id'], 
                                 'key': 'country',
                                 'type': 'is_in',
                                 'value': country})
                break

## Importing the processed data into sqlite
Running the data.py code generates 5 .csv files. The header of the CSV files were removed, as they would cause issues when being imprted. 5 tables were created according to the schemas and each files was imported to the associated table. the resulting files and their size is:

File | size
--- | ---
edmonton_canada.osm | 788,945 KB
OSM_Edmonton.db | 443,644 KB
nodes.csv | 314,580 KB
nodes_tags.csv | 21,580 KB
wayz.csv | 28,285 KB
ways_nodes.csv | 103,120 KB
ways_tags.csv | 25,721 KB

## Some statistics about dateset
### Total number of Nodes:
```SQL
sqlite> SELECT COUNT(*) AS TotalNodes FROM Nodes;
TotalNodes
----------
3718499 
```

### Total number of Ways:
```SQL
sqlite> SELECT COUNT(*) as TotalWays FROM Ways;
TotalWays
----------
469009
```

### Number of Nodes with tags:
```SQL
sqlite> SELECT COUNT(DISTINCT(ID)) AS NodesWithTag FROM NodeTags;
NodesWithTag
------------
157647
```

### Number of unique users:
```SQL
sqlite> SELECT COUNT(DISTINCT(NodeUniWay.UserID)) as TotalUsers FROM (SELECT UserID FROM Nodes UNION SELECT UserID FROM Ways) as NodeUniWay;
TotalUsers
----------
670
```

### Number of ways for each city
Here only the 10 cities with the most ways are chosen. This partly shows how big is each city. (City names are truncated, otherwise they are correct in the database)
```SQL
sqlite> SELECT WayTags.Value as City, COUNT(*) as TotalWays
   ...> FROM Ways, WayTags
   ...> WHERE Ways.ID = WayTags.ID
   ...> AND WayTags.Key = 'city'
   ...> GROUP BY WayTags.Value
   ...> ORDER BY TotalWays DESC
   ...> LIMIT 10;
City        TotalWays
----------  ----------
Edmonton    7697
Strathcona  3026
Leduc Coun  2077
Parkland C  1911
Sturgeon C  1684
Wetaskiwin  1684
St. Albert  1588
Lac Ste. A  836
Spruce Gro  810
Barrhead    779
```

### The issues identified in the database (fixme tag)
#### Node tag issues
From the around 160,000 nodes that have tags, about 1100 have 'fixme' key. The most common required fix that is identified in the OSM database is the housenumber. This is likely done programmatically. A similar issue was observed through the data audit when checking the street name tags. 
```SQL
sqlite> SELECT Value AS FixmeValue, COUNT(*)
   ...> FROM NodeTags
   ...> WHERE Key = "fixme"
   ...> GROUP BY Value
   ...> ORDER BY COUNT(*) DESC
   ...> LIMIT 5;
FixmeValue                  COUNT(*)
--------------------------  ----------
addr:housenumber to revise  1014
Place type may not be vali  94
identify extracted resourc  9
continue                    5
Feature may not be valid    2
```

#### Way tag issues
```SQL
sqlite> SELECT Value AS FixmeValue, COUNT(*)
   ...> FROM WayTags
   ...> WHERE Key = "fixme"
   ...> GROUP BY Value
   ...> ORDER BY COUNT(*) DESC
   ...> LIMIT 5;
FixmeValue                      COUNT(*)
------------------------------  ----------
could be dunes or raised_beach  91
sport type is not known         16
name                            4
Clareview station is not reall  2
merge streets to this           2
```

#### The user associated with most nodes that contain fixme tag
From the result we can see that most tags with fixme are associated with one user. It is likely that this user is using some form of automated tool to identify the fixme tags.
I am not sure if the User associated with a node is tha last person that updated it, or the person who created the node. Therefore, it is not cleat if the Users below tagged the 'fixme' key or created the node that was later tagged for fix. 
```SQL
sqlite> SELECT User, COUNT(*) AS Num
   ...> FROM (SELECT User FROM WayTags JOIN Ways
   ...> ON WayTags.ID = Ways.ID WHERE Key = "fixme"
   ...> UNION ALL SELECT User FROM NodeTags JOIN Nodes
   ...> ON NodeTags.ID = Nodes.ID WHERE Key = "fixme") AS UserUnion
   ...> GROUP BY User
   ...> ORDER BY Num DESC
   ...> LIMIT 10;
User        Num
----------  ----------
PierZen     1011
charrois    196
Viajero Pe  12
MuzikMachi  11
Sundance    6
Angus C     5
mesowhite   3
phillipsjk  3
yoltia      3
Mesowhite   2
```

## Other ideas about the datasets
### Confidence level on validity of data
The OSM is user generated, so besides the different representation of same infromation (St vs Street), there is always some inputs that are incorrect.

It would be a good idea to include a percentage number showing the accuracy of the information in that 'node' or 'way'. This has two benefits. First, anyone who is developing applications based on the OSM data can decide which information they want to trust and which information they want to skip. Second, people who want to contribute to the database can identify which infroamtion is not accurate and requires review.

Ideally, this process would be automated. Similar to the way the Street types were audited, various tags can be audited and based on the number of incostencies found, a confidence number will be given to each node or way.  
Of course, this process requires the data be compared to a separate database. In the case of street type, the data base was basically a python list that identifies the valid street types.
I belielve it is possible to query the data against google maps database or foursquare database and see how accurate is a specific node or way 

### Anonymous GPS data collection for automated extraction of map features
Another idea to improve and automate the process of improving map features is to collect anonymous data from OSM users, possilibly through free mobile applications that utilize OSM data.  
When enough GPS data is avialabe for a street, that data can be used to identify the speed limit, direction of street, school zones and its hours, on-street parking limitations, etc. These information seems to be missing from OSM and would significantly improve its use in mobile navigation.
